In [1]:
from tqdm import tqdm, trange
import json
from collections import namedtuple, defaultdict
import pandas as pd
import numpy as np
import torch
import os
from gensim.parsing.preprocessing import strip_non_alphanum
from gensim.models.word2vec import Word2Vec
import random
from scipy.sparse import csr_matrix
import dgl

import pickle as pkl
import sys
sys.path.append('/mnt/nfs/zhangtl/utils')
from util import myout

Using backend: pytorch


## load

In [2]:
dataset = 'math'
df = pd.read_csv(f'../../../01_process/data/{dataset}/Papers.csv') # 5353904
df.dropna(subset=['p_id', 'year'], inplace=True)
df.fillna({'refs': '', 'a_ids': '', 'v_id': '', 'fids': '', 'abst': ''}, inplace=True)
df

,Unnamed: 0,title,p_id,year,ncites,refs,a_ids,v_id,fids,abst
0,1793859,Appendice all’articolo,2.473352e+09,1858.0,2,,2675796970,177770778,"1,31,4",
1,693758,Avviso dei Compilatori,2.020185e+09,1858.0,0,,"2700887317,2640568158,2647493715,2698016866",177770778,"191,31,1",
2,135352,"Beiträge Zur Géometrie der lage, vonDr. Georg ...",1.574345e+09,1858.0,2,2125588138,665515178,177770778,"19,1,131",
3,780899,Constructions du centre de courbure de la cour...,2.030261e+09,1858.0,0,,2314525361,177770778,"19,131,1",
4,270824,Di una nota del barone plana. casi particolari...,1.971452e+09,1858.0,0,,2634759845,177770778,"1,19,131",
...,...,...,...,...,...,...,...,...,...,...
2390100,2384802,Towards Knowledge Warehousing: Application to ...,3.037733e+09,2021.0,0,,"2306801064,2769702725",118326976,"1,1336,271,620",
2390101,1921404,Unifying and Generalizing Methods for Removing...,2.617653e+09,2021.0,10,"131298435,612972800,1500618125,1533942137,1536...","2099113133,2252142780",23408671,"6,271,2182,1384,175,2909,1,3354,649,4302",
2390102,2384718,Varying Coefficient Panel Data Model with Inte...,3.037696e+09,2021.0,1,,"2150863536,2106876108,2143830904,2276330680",23408671,"6,1,7820,105",
2390103,2382415,Virtual complete intersections in P1×P1,3.036669e+09,2021.0,0,"1483218536,1491346501,1498202005,1594371819,19...","2945802986,2945171878,2946734732,2946087421",4805881,"4277,11163,855,1,197,6706,33432,82,5968",


In [3]:
start_year, end_year = 2000, 2022
drop_id = df[(df.year < start_year) | (df.year >= end_year)].index
print(f'Drop {len(drop_id)} rows') # 559749
papers = df.drop(drop_id) # 3547589
papers

Drop 792450 rows


,Unnamed: 0,title,p_id,year,ncites,refs,a_ids,v_id,fids,abst
792662,2327963,"""A Distributed Computing Environment for Genet...",3.011053e+09,2000.0,0,,"3033714930,1888566757,191220263,2284272671",106296714,"1109,180,7217,4329,8579,7,3915",
792663,2343004,"""Au bonheur des dames"" ou la leçon de commerce...",3.015662e+09,2000.0,0,,2659952957,18902827,"3,131,1",
792664,61087,"""Counterexamples"" to the harmonic Liouville th...",9.532570e+08,2000.0,8,"1509577866,1981284421,2020239290,2026830775,20...",2473743554,111346810,"11705,922,876,848,1702,19,848,848,2457,1",
792665,139025,"""La Odisea de Querer ser Centrista"": contrasti...",1.580039e+09,2000.0,1,,2702200331,57950554,"2633,620,271,16248,3760,1",
792666,442843,"""M""-estimation Under a Two-Sample Semiparametr...",1.991266e+09,2000.0,9,"8090641,1528009881,1967573895,1992147656,20181...",2155192949,72071028,"1,6,2839,105,1734,625,1516,3632,10602,4209,4759",
...,...,...,...,...,...,...,...,...,...,...
2390100,2384802,Towards Knowledge Warehousing: Application to ...,3.037733e+09,2021.0,0,,"2306801064,2769702725",118326976,"1,1336,271,620",
2390101,1921404,Unifying and Generalizing Methods for Removing...,2.617653e+09,2021.0,10,"131298435,612972800,1500618125,1533942137,1536...","2099113133,2252142780",23408671,"6,271,2182,1384,175,2909,1,3354,649,4302",
2390102,2384718,Varying Coefficient Panel Data Model with Inte...,3.037696e+09,2021.0,1,,"2150863536,2106876108,2143830904,2276330680",23408671,"6,1,7820,105",
2390103,2382415,Virtual complete intersections in P1×P1,3.036669e+09,2021.0,0,"1483218536,1491346501,1498202005,1594371819,19...","2945802986,2945171878,2946734732,2946087421",4805881,"4277,11163,855,1,197,6706,33432,82,5968",


## build graph

In [5]:
dataset = 'math'
p_id2emb = pkl.load(open(f'../../../01_process/data_papers/{dataset}/p_id2emb.pkl', 'rb'))
myout(p_id2emb)
def update_idx(idx, dic, cnt, feats, rel, feat_dim, no_emb):
    if idx not in dic:
        dic[idx] = cnt
        cnt += 1
        if rel == 0:
            try:
                feats.append(torch.from_numpy(p_id2emb[idx]).to(torch.float32))
            except:
                feats.append(torch.rand(feat_dim).to(torch.float32)-1)
                no_emb += 1
        elif rel == 1:
            feats.append(torch.rand(feat_dim).to(torch.float32)+1)
        elif rel == 2:
            feats.append(torch.rand(feat_dim).to(torch.float32)+2)
        elif rel == 3:
            feats.append(torch.rand(feat_dim).to(torch.float32))
    return dic, cnt, feats, no_emb

p_id2emb : len=2388421, dict([2473352429: [-2.86228880e-02 -1.44007251e-01  2.44896144e-01  2.24655271e-01
 -3.63689661e-03 -4.68410522e-01  1.58079237e-01 -1.85831755e-01
 -1.35813616e-02  1.20756269e-01 -1.31523922e-01  1.14154555e-01
  1.97942764e-01 -8.98324475e-02 -1.49471909e-01  7.10590929e-02
  5.23683056e-02 -7.22697228e-02 -3.64452600e-03 -1.76061302e-01
  1.77970618e-01  1.37115508e-01 -1.40968353e-01 -1.03638828e-01
  1.07044868e-01  1.13885462e-01 -1.37735605e-01 -1.50651425e-01
 -2.21623704e-02 -2.76831925e-01 -1.11679882e-01 -1.52502835e-01
 -1.60274401e-01 -2.06519008e-01  1.09443754e-01  2.26094797e-01
  1.06575564e-01 -1.01515494e-01  2.18097568e-01  3.49558592e-01
  2.34268606e-03 -1.00658625e-01  2.86409438e-01  2.17967555e-01
 -6.60552084e-02  2.02980071e-01  1.93566978e-02 -1.64073467e-01
 -1.26613304e-05  1.30400598e-01  2.44674489e-01  1.85491547e-01
  2.38537192e-02 -2.11097986e-01  1.83463112e-01  1.46224499e-01
 -7.12506771e-02 -3.34030032e-01  6.82454407e-02

In [6]:
start_year, end_year = 2000, 2022
feat_dim = 128

id2nid, cnt, no_emb = {}, 0, 0
lst, feats = [], []
for i in trange(len(papers)):
    ts = int(papers['year'].iloc[i])
    p_id = int(papers['p_id'].iloc[i])
    refs = papers['refs'].iloc[i]
    
    id2nid, cnt, feats, no_emb = update_idx(p_id, id2nid, cnt, feats, 0, feat_dim, no_emb)
    if len(refs)>0:
        rlst = refs.split(',')
        for ref in rlst:
            if ref != '':
                ref = int(ref)
                id2nid, cnt, feats, no_emb = update_idx(ref, id2nid, cnt, feats, 0, feat_dim, no_emb)
                lst.append((id2nid[p_id], id2nid[ref], 0, ts))
        
feat = torch.stack(feats)
src = torch.tensor([item[0] for item in lst])
tgt = torch.tensor([item[1] for item in lst])
rel = torch.tensor([item[2] for item in lst])
tsp = torch.tensor([item[3] for item in lst])
myout(feat, src, tgt, rel, tsp)

100%|██████████| 1596676/1596676 [03:34<00:00, 7446.81it/s]


feat : shape=torch.Size([8163183, 128])
tensor([[ 0.1281, -0.1832,  0.5622,  ..., -0.1985,  0.4194,  0.0648],
        [-0.1986, -0.3023,  0.2742,  ..., -0.0177,  0.1201, -0.2316],
        [ 0.2232,  0.1367,  0.1659,  ...,  0.0167,  0.4633,  0.0891],
        ...,
        [-0.6144, -0.0220, -0.8504,  ..., -0.2454, -0.0191, -0.4750],
        [-0.7049, -0.9819, -0.8327,  ..., -0.9985, -0.0649, -0.7223],
        [-0.4087, -0.7891, -0.1075,  ..., -0.5051, -0.7883, -0.1023]])
src : shape=torch.Size([30181076]), tensor([      2,       2,       2,  ..., 8163178, 8163178, 8163178])
tgt : shape=torch.Size([30181076]), tensor([      3,       4,       5,  ..., 8163182, 7379241, 4327430])
rel : shape=torch.Size([30181076]), tensor([0, 0, 0,  ..., 0, 0, 0])
tsp : shape=torch.Size([30181076]), tensor([2000, 2000, 2000,  ..., 2021, 2021, 2021])


In [7]:
graph = dgl.graph((src, tgt), num_nodes=len(feat))
graph.ndata['feat'] = feat

nid2id = {vv: kk for kk, vv in id2nid.items()}
graph.ndata['raw_nid'] = torch.arange(len(feat))

graph.edata['rel'] = rel
graph.edata['ts'] = tsp
graph

Graph(num_nodes=8163183, num_edges=30181076,
      ndata_schemes={'feat': Scheme(shape=(128,), dtype=torch.float32), 'raw_nid': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'rel': Scheme(shape=(), dtype=torch.int64), 'ts': Scheme(shape=(), dtype=torch.int64)})

In [8]:
dgl.save_graphs(f'../data/{dataset}/graph.bin', [graph])

In [9]:
json.dump(id2nid, open(f'../data/{dataset}/id2nid.json', 'w'))

## gen cites

In [10]:
cites = {}
print(start_year, end_year)
for year in range(start_year, end_year):
    cites[year] = defaultdict(int)

papers.fillna({'refs': ''}, inplace=True)
for i in trange(len(papers)):
    year = int(papers['year'].iloc[i])
    refs = papers['refs'].iloc[i]
    if len(refs)>0:
        rlst = refs.split(',')
        rlst = [int(item) for item in rlst]
        for ref in rlst:
            if ref != '':
                cites[year][ref] += 1

2000 2022


100%|██████████| 1596676/1596676 [00:51<00:00, 30780.15it/s]


In [11]:
tsp = graph.edata['ts']
ts_vals, ts_cuts = np.unique(tsp.numpy(), return_index=True)
ts_cuts = list(ts_cuts) + [len(tsp.numpy())]

num_ts = len(ts_vals)
ts_infos = np.stack([ts_vals, ts_cuts[0:num_ts], ts_cuts[1:num_ts+1]]).transpose()
myout(ts_cuts, ts_vals, ts_infos)

ts_cuts : len=23, list([0, 474218, 978765, ..., 28072017, 30178481, 30181076])
ts_vals : shape=(22,), [2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017 2018 2019 2020 2021]
ts_infos : shape=(22, 3)
[[    2000        0   474218]
 [    2001   474218   978765]
 [    2002   978765  1537738]
 [    2003  1537738  2184804]
 [    2004  2184804  2885592]
 [    2005  2885592  3694395]
 [    2006  3694395  4631370]
 [    2007  4631370  5686967]
 [    2008  5686967  6826344]
 [    2009  6826344  8141764]
 [    2010  8141764  9486968]
 [    2011  9486968 10956317]
 [    2012 10956317 12611444]
 [    2013 12611444 14443862]
 [    2014 14443862 16395309]
 [    2015 16395309 18452737]
 [    2016 18452737 20570161]
 [    2017 20570161 22852682]
 [    2018 22852682 25178714]
 [    2019 25178714 28072017]
 [    2020 28072017 30178481]
 [    2021 30178481 30181076]]


In [12]:
labels = {}
nid2id = {v:k for k,v in id2nid.items()}
for year in range(start_year, end_year):
    left, right = ts_infos[np.where(ts_infos[:, 0]==year)[0][0], 1:]
    nids = graph.edges()[0][left:right].unique().tolist()
    ids = [nid2id[nid] for nid in nids]
    
    pdf = pd.DataFrame({'id': ids, 'nid': nids})
    tbar = trange(year+1, end_year, desc=str(year))
    for yy in tbar:
        cdf = pd.DataFrame({'id': list(cites[yy].keys()), str(yy): list(cites[yy].values())})
        cdf[str(yy)] = cdf[str(yy)].astype('float32')
        
        pdf = pd.merge(pdf, cdf, how='left', on='id')
        tbar.set_postfix(year=year, pdf=len(pdf))
    pdf.fillna(0, inplace=True)
    labels[year] = pdf
    

2020: 100%|██████████| 1/1 [00:00<00:00, 96.56it/s, pdf=69081, year=2020]
2021: 0it [00:00, ?it/s]


In [13]:
labels[2005]

,id,nid,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,2067451100,135481,1.0,0.0,1.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2001816150,141991,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2168206261,154958,2.0,3.0,1.0,2.0,3.0,2.0,0.0,1.0,2.0,1.0,2.0,1.0,0.0,0.0,2.0,0.0
3,2040785660,169304,0.0,1.0,0.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,1.0,0.0,3.0,1.0,1.0,0.0
4,2132105090,171116,2.0,6.0,10.0,3.0,13.0,13.0,13.0,19.0,18.0,17.0,15.0,19.0,20.0,31.0,15.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44678,2215102431,1555124,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44679,1492638500,1555131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44680,1929114195,1555188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44681,1567499869,1555193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
pkl.dump(labels, open(f'../data/{dataset}/labels.pkl', 'wb'))

## cum log labels

In [15]:
def cumulative_log(df):
    colsn = list(df.columns)
    for i in range(3, len(colsn)):
        df[colsn[i]] += df[colsn[i-1]]
    df.iloc[:, 2:] = np.log(df.iloc[:, 2:] + 1)
    return df

labels_cum_log = {}
for year in range(start_year, end_year-2): # 2000, 2020
    labels_cum_log[year] = cumulative_log(labels[year])
labels_cum_log[end_year-2] = labels[end_year-2]
print(len(labels_cum_log))
labels_cum_log[2005]

21


,id,nid,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,2067451100,135481,0.693147,0.693147,1.098612,1.609438,1.609438,1.945910,1.945910,1.945910,1.945910,1.945910,1.945910,2.079442,2.079442,2.079442,2.079442,2.079442
1,2001816150,141991,0.000000,0.000000,0.693147,0.693147,1.098612,1.098612,1.098612,1.098612,1.098612,1.098612,1.098612,1.386294,1.386294,1.386294,1.386294,1.386294
2,2168206261,154958,1.098612,1.791759,1.945910,2.197225,2.484907,2.639057,2.639057,2.708050,2.833213,2.890372,2.995732,3.044523,3.044523,3.044523,3.135494,3.135494
3,2040785660,169304,0.000000,0.693147,0.693147,0.693147,1.098612,1.386294,1.386294,1.791759,1.945910,1.945910,2.079442,2.079442,2.397895,2.484907,2.564949,2.564949
4,2132105090,171116,1.098612,2.197225,2.944439,3.091043,3.555348,3.871201,4.110874,4.382027,4.584968,4.744932,4.867535,5.003946,5.129899,5.298317,5.370638,5.370638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44678,2215102431,1555124,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
44679,1492638500,1555131,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
44680,1929114195,1555188,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
44681,1567499869,1555193,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [16]:
pkl.dump(labels_cum_log, open(f'../data/{dataset}/labels_cum_log.pkl', 'wb'))